In [1]:
import os

In [3]:
from pathlib import Path
from saffron.io import data_io
from saffron.data import datasets, data_processing
from saffron.models import torch_models
import matplotlib.pyplot as plt
import numpy as np

In [10]:
microglia_im_path = "/mmfs1/gscratch/cheme/nlsschim/data/cross_species/ferret"

In [11]:
images = data_io.load_images_from_directory(microglia_im_path)

INFO:saffron.io.data_io:Successfully loaded 138 images from /mmfs1/gscratch/cheme/nlsschim/data/cross_species/ferret


In [12]:
for i, im in enumerate(images):
    print(im.shape)

(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 512)
(21, 512, 